In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from explorer import Explorer

In [ ]:
e = Explorer()

In [ ]:
for i in range(20):
    e.explore()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from explorer import Explorer
e = Explorer()

In [ ]:
len(e.exp_backend.exp_store)

In [ ]:
e.remove_redundant_experiences()

In [ ]:
e.remove_redundant_experiences()